In [ ]:
import sys
import heapq


def main():
    inf = 10**30
    penalty = 10**18
    t = int(input())
    for _ in range(t):
        n, m = map(int, input().split())
        a = list(map(int, input().split()))
        b = list(map(int, input().split()))
        q, r = divmod(n, m)
        source = 0
        sink = n + m + 1
        size = sink + 1
        graph = [[] for _ in range(size)]
        def add_edge(u, v, cap, cost):
            graph[u].append([v, cap, cost, len(graph[v])])
            graph[v].append([u, 0, -cost, len(graph[u]) - 1])

        for i in range(n):
            add_edge(source, i + 1, 1, 0)

        for j in range(m):
            group = n + 1 + j
            add_edge(group, sink, q, 0)
            add_edge(group, sink, 1, penalty)

        for i in range(n):
            ai = a[i]
            u = i + 1
            for j in range(m):
                bj = b[j]
                cost = (bj - ai % bj) % bj
                v = n + 1 + j
                add_edge(u, v, 1, cost)

        potential = [0] * size
        flow = n
        total_cost = 0

        while flow:
            dist = [inf] * size
            dist[source] = 0
            prev_v = [-1] * size
            prev_e = [-1] * size
            heap = [(0, source)]

            while heap:
                d, u = heapq.heappop(heap)
                if d != dist[u]:
                    continue
                for i, e in enumerate(graph[u]):
                    if e[1] == 0:
                        continue
                    v, _, cost, _ = e
                    nd = d + cost + potential[u] - potential[v]
                    if nd < dist[v]:
                        dist[v] = nd
                        prev_v[v] = u
                        prev_e[v] = i
                        heapq.heappush(heap, (nd, v))

            if dist[sink] == inf:
                break

            for i in range(size):
                if dist[i] < inf:
                    potential[i] += dist[i]

            add = flow
            v = sink
            while v != source:
                u = prev_v[v]
                e = graph[u][prev_e[v]]
                add = min(add, e[1])
                v = u

            flow -= add
            total_cost += add * potential[sink]

            v = sink
            while v != source:
                u = prev_v[v]
                e = graph[u][prev_e[v]]
                e[1] -= add
                graph[v][e[3]][1] += add
                v = u
                
        print(total_cost - penalty * r)


if __name__ == "__main__":
    main()